# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [1]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [2]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print(classes)

['Sit', 'Back', 'Right', 'Crouch', 'Left', 'Frog', 'Stand', 'Knee', 'HeadBack', 'StandInit', 'Belly']


In [3]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename, 'rb'))
    target = [i] * len(data)
    
    data = np.array(data)
    target = np.array(target)
    
    return data, target

In [23]:
# load all the data
all_data = []
all_target = []
# TODO

for i in range(len(classes)):
    pose_data = load_pose_data(i)
    for j in range(len(pose_data[0])):
        all_data.append(pose_data[0][j])
        all_target.append(pose_data[1][j])
    
all_data = np.array(all_data)
all_target = np.array(all_target)

print(all_data.shape, all_target.shape)
print('total number of data', len(all_data))

print(type(all_data[0][0]))

(222, 10) (222,)
total number of data 222
<class 'numpy.float64'>


In [5]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [6]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [7]:
clf.fit(all_data[0:], all_target[0:])

SVC(C=100.0, gamma=0.001)

### predicting

In [8]:
clf.predict(all_data[-1:]), all_target[-1:]

(array([10]), array([10]))

In [9]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [10]:
expected = []
predicted = []

predicted = clf.predict(all_data)

evaluate(all_target, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        26
           1       1.00      1.00      1.00        23
           2       1.00      0.91      0.95        11
           3       0.97      1.00      0.98        30
           4       1.00      1.00      1.00        20
           5       1.00      1.00      1.00        10
           6       1.00      1.00      1.00        11
           7       1.00      1.00      1.00        10
           8       1.00      1.00      1.00        10
           9       1.00      1.00      1.00        52
          10       1.00      1.00      1.00        19

    accuracy                           1.00       222
   macro avg       1.00      0.99      0.99       222
weighted avg       1.00      1.00      1.00       222


Confusion matrix:
[[26  0  0  0  0  0  0  0  0  0  0]
 [ 0 23  0  0  0  0  0  0  0  0  0]
 [ 0  0 10  1  0  0  0  0  0  0  0]
 [ 0  0  0 30  0  0  0  0  0  0  0]
 

## 4. Evaluate on the test data

In [11]:
expected = []
predicted = []
test_data = []

for index in permutation:
    expected.append(all_target[index])
    test_data.append(all_data[index])

predicted = clf.predict(test_data)
evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        26
           1       1.00      1.00      1.00        23
           2       1.00      0.91      0.95        11
           3       0.97      1.00      0.98        30
           4       1.00      1.00      1.00        20
           5       1.00      1.00      1.00        10
           6       1.00      1.00      1.00        11
           7       1.00      1.00      1.00        10
           8       1.00      1.00      1.00        10
           9       1.00      1.00      1.00        52
          10       1.00      1.00      1.00        19

    accuracy                           1.00       222
   macro avg       1.00      0.99      0.99       222
weighted avg       1.00      1.00      1.00       222


Confusion matrix:
[[26  0  0  0  0  0  0  0  0  0  0]
 [ 0 23  0  0  0  0  0  0  0  0  0]
 [ 0  0 10  1  0  0  0  0  0  0  0]
 [ 0  0  0 30  0  0  0  0  0  0  0]
 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [14]:
import pickle
ROBOT_POSE_CLF = "robot_pose.pkl"
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))
print(clf)

SVC(C=100.0, gamma=0.001)


Then, in the application we can load the trained classifier again.

In [20]:
clf2 = pickle.load(open(ROBOT_POSE_CLF, 'rb'))
clf2.predict(all_data[-1:]), all_target[-1:]

(array([10]), array([10]))